<a href="https://colab.research.google.com/github/maddran/lordstanleyscup/blob/main/code/LSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r lordstanleyscup/
!git clone https://github.com/maddran/lordstanleyscup

from lordstanleyscup.code.helpers import *

Cloning into 'lordstanleyscup'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 58 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [2]:
SCWINNER = "COL"

sched_df = get_schedule()
draft_df = pd.read_csv('/content/lordstanleyscup/draft_data/Stanley Cup - 22_23 - Draft.csv')
draft_df.columns = ['round', 'pick', 'owner', 'team']

draft_LU = dict(zip(draft_df.team, draft_df.owner))

In [3]:
score_df = sched_df

score_df['cupGame'] = 0
score_df['cupTeam'] = None


In [4]:
import pandas as pd
import numpy as np

def next_cup_game(df, cupteam):
  return min((df[df.cupTeam.isnull()].homeTeam == cupteam).idxmax(),
    (df[df.cupTeam.isnull()].awayTeam == cupteam).idxmax())

def get_winner(row):
  if row.homeScore > row.awayScore:
    return row.homeTeam
  elif row.homeScore < row.awayScore:
    return row.awayTeam
  else:
    return None

def score_season(df, cupteam = SCWINNER):

  if df.cupTeam.last_valid_index() is None:
    last_idx = 0
  else:
    last_idx = df.cupTeam.last_valid_index()
    cupteam = df.loc[last_idx,'cupTeam']

  next_idx = next_cup_game(df[(df.homeScore+df.awayScore)>0], cupteam)
  if(next_idx <= last_idx+1):
    return

  df.loc[next_idx, 'cupGame'] = 1
  df.loc[last_idx:next_idx-1, 'cupTeam'] = cupteam
  cupteam = get_winner(df.loc[next_idx])
  df.loc[next_idx, 'cupTeam'] = cupteam

  score_season(df, cupteam)


In [5]:
score_season(score_df, SCWINNER)

for p in ['home', 'away', 'cup']:
  score_df[f'{p}Owner'] = score_df[f'{p}Team'].apply(lambda x: draft_LU.get(x)).fillna('')

In [74]:
points_df = score_df[(score_df.cupGame == 1) & ((score_df.homeScore + score_df.awayScore) != 0)]

points_summary = points_df.groupby(['cupOwner']).cupGame.sum().reset_index()
points_summary.columns = ['owner', 'points']

cup_games_summary_home = points_df.groupby(['homeOwner','homeTeam']).cupGame.sum().reset_index()
cup_games_summary_home.columns = ['owner', 'team', 'cupGames']
cup_games_summary_away = points_df.groupby(['awayOwner','awayTeam']).cupGame.sum().reset_index()
cup_games_summary_away.columns = ['owner', 'team', 'cupGames']

cup_games_summary = pd.concat([cup_games_summary_away, cup_games_summary_home]).groupby(['owner']).cupGames.sum().reset_index()

full_summary = cup_games_summary.merge(points_summary, how = 'outer')
full_summary

,owner,cupGames,points
0,A,2,2.0
1,E,3,1.0
2,M,1,NaN


In [76]:
snake = points_df.groupby(['gameDate','cupOwner','cupTeam']).cupGame.sum().groupby(level=1).cumsum().reset_index()
snake.columns = ['gameDate', 'owner', 'cupTeam', 'points']
snake

,gameDate,owner,cupTeam,points
0,20221012,E,COL,1
1,20221013,A,CGY,1
2,20221015,A,CGY,2
